In [ ]:
searchTerm='#trump'

//Enter the search term here.

In [ ]:
library(dplyr)

In [ ]:
library(twitteR)

In [ ]:
library(sqldf)

In [ ]:
library(ggmap)

In [ ]:
consumer_key <- ''
consumer_secret <- ''
access_token <- ''
access_secret <- ''

In [ ]:
setup_twitter_oauth(consumer_key, consumer_secret, access_token, access_secret)

//Set up twitter OAuth

In [ ]:
db <- dbConnect(SQLite(), dbname='dic')

In [ ]:
register_db_backend(db)

//Creates a database file named 'dic'. It is included in the folder.

In [ ]:
count=0

In [ ]:
tweets = searchTwitter(searchTerm,n=100)
store_tweets_db(tweets)

data = twListToDF(tweets)
min = data[which.min(data$id),]

//The above code searches twitter for 100 tweets

In [ ]:
maxid = as.numeric(min[[8]])-100

//The above code finds the minimum id of the above batch so that it acts as the maxid for the next batch.

In [ ]:
while(count!=150){
    tweets = searchTwitter(searchTerm,n=100, maxID=maxid)
    store_tweets_db(tweets)
    data = twListToDF(tweets)
    min = data[which.min(data$id),]
    maxid = as.numeric(min[[8]])-100
    count=count+1
    print(count)
    }

//The above code seaches twitter for 100 tweets, 150 times and storing the tweets in the database at the same time. Total of 15000 tweets. 
//Due to API limits the total is kept 15000. It can be increased or the loop can be run over again as per convinience.

In [ ]:
from_db = load_tweets_db()

In [ ]:
data = twListToDF(from_db)

In [ ]:
screenname = data[['screenName']]

//Above lines of code load the tweets from the database and convert them to Dataframe. Then get the screen name of each tweet from it.

In [ ]:
i=1
user=''
while(i<20000)
    {
    user = c(user,screenname[i])
    i=i+1
}


//Above code makes user variable containing all the screen names to pass it on to the next function at a time. i.e. lookUpUsers() to get the info on the user having the screen name.

In [ ]:
user1 = lookupUsers(user)

In [ ]:
user2 = twListToDF(user1)

In [ ]:
user4 = user2[!user2$location == '', ]

//Above code deletes all the users having nothing as location.

In [ ]:
cat(user4$location,file='/Users/anandpopat/desktop/b.csv',sep='\n',append=TRUE)

//Above code writes the data to file to save it.

In [ ]:
address<-readLines('/Users/anandpopat/desktop/b.csv', n = -1)

//Reads the data saved in the above line to the file. Can be skipped but just to keep a backup.

In [ ]:
df3=''

In [ ]:
i=2

In [ ]:
geo_reply=''
geo_reply<-as.numeric(geocode(address[i]))
df3=data.frame(geo_reply[1],geo_reply[2])
i=i+1
print(i)

In [ ]:
while(i<500){
    geo_reply=''
    geo_reply<-as.numeric(geocode(address[i]))
    df1=data.frame(geo_reply[1],geo_reply[2])
    df3=rbind(df3,df1)
    i=i+1
    print(i)
}

//Above lines of code get the geocode which is a google API function to get the lat-lon pair of location of the users we collected in the previous steps. 
//Due to API limits and other errors the batch processing is done in the size of 500s. Can be increased as per convinience.

In [ ]:
write.csv(file='/Users/anandpopat/desktop/location1.csv',x=df3)

In [ ]:
read=read.csv('/Users/anandpopat/desktop/location1.csv')

//The data is written and read from 9 location files which are included in the folder. Each file has 500 locations except location2.csv which has 1000 locations. The path has to be changed for each file. Again the processing is done in batches of 500s. To avoid writing data to different files, the limit can be increased.

//Data written to file and read again just for backup purposes.

In [ ]:
j=2

In [ ]:
while(j<500){
    
    loc = as.numeric(read[j,2:3])
    state=revgeocode(loc,output ='more')
    state1=toString(state$administrative_area_level_1)
    if(toString(state$country)=='United States')
    {
        if(state1!="")
        {
            cat(state1,file='/Users/anandpopat/desktop/state1.csv',sep='\n',append=TRUE)
        }
    }
    j=j+1
    print(j)
}

//Above lines of code reverse geocode the lat-lon pair obtained in the previous steps to get a concrete address and then filter out any address not in US. Only the states obtained from the results are saved and written to a file for backup purposes.
//Again due to API limits the batch is processed in 500s. Can be increased.

In [44]:
geocodeQueryCheck()

2421 geocoding queries remaining.


//Just to check how many API calls still left.

In [ ]:
read_state<-readLines('/Users/anandpopat/desktop/state1.csv', n = -1)

In [ ]:
read_state=data.frame(read_state)

//Reads the states we saved in the previous step and convert it to dataframe

In [ ]:
grp1 <- group_by(read_state,read_state)
(grp2 <- summarize(grp1,count=n()))

//Above code groups the states and gives the count of its occurence.

In [ ]:
write.csv(file='/Users/anandpopat/desktop/final_list1.csv',x=grp2)

//Saves the states with their count of occurence in a file.

In [ ]:
final_list1 = read.csv("/Users/anandpopat/desktop/final_list1.csv")
final_list1$read_state <- as.character(final_list1$read_state)
final_list1$count = as.numeric(final_list1$count)
final_list1 = final_list1[final_list1$read_state != "Alaska", ]
final_list1 = final_list1[final_list1$read_state != "Hawaii", ]


//Makes a proper data frame for further processing from the data saved in the previous step. Also removes alaska and hawaii from the result to obtain a 'united' united states map. :)

In [ ]:
for (i in 1:nrow(final_list1)) {
  latlon = geocode(final_list1[i,2])
  final_list1$lon[i] = as.numeric(latlon[1])
  final_list1$lat[i] = as.numeric(latlon[2])
}

//Above lines of code get the lat-lon pair of the states of US obtained in the previous step for plotting on the map.

In [ ]:
write.csv(file='/Users/anandpopat/desktop/final_list_loc1.csv',x=final_list1)

In [ ]:
final_list1=read.csv(file='/Users/anandpopat/desktop/final_list_loc1.csv')

In [ ]:
state_count = data.frame(final_list1$count, final_list1$lon, final_list1$lat)
colnames(state_count) = c('count','lon','lat')


//Set a proper data frame from the data saved in the file i.e count, state, lat and lon.

In [ ]:
usa = as.numeric(geocode("United States"))
mymap = ggmap(get_googlemap(center=usa, scale=2, zoom=4), extent="normal")

//Get the map of USA from ggmap.

In [ ]:
mymap + geom_point(aes(x=final_list1$lon, y=final_list1$lat), data=state_count, col="orange", alpha=0.4, size=state_count$count*0.1) + scale_size_continuous(range=range(state_read$count))

//Add orange blobs to the map of USA showing the density of the tweets coming from that area.
//State count in multiplied by 0.1 to adjust the size of the blob. Can be changed to suit the needs.

REFERENCES:
DIC Lab1.pdf
twitteR Vignette.
https://blog.dominodatalab.com/geographic-visualization-with-rs-ggmaps/
https://www.r-bloggers.com/mapping-twitter-followers-in-r/
https://www.r-bloggers.com/dplyr-example-1/